In [ ]:
import numpy as np
from music21 import *

def chorale_to_inputs(s,i):
    inputs = []
    for j in range(16):
        inputs.append([133])
    for n in s.parts[i].flat.notes:
#         if n.isNote:
#             print(n.nameWithOctave, int(4 * n.quarterLength), n.offset)
#         else:
#             print(n.pitchNames, int(4 * n.quarterLength), n.offset)
        #print(n.expressions)
        if n.isNote:
            inputs.append([n.pitch.midi])
            note_type.add(frozenset([n.pitch.midi]))
        elif n.isChord:
            inputs.append([_note.pitch.midi for _note in n])
            note_type.add(frozenset([_note.pitch.midi for _note in n]))
        quarter_note_length = n.quarterLength
#         if l == 0:
#             l = 1
        for j in range(1, int(4 * quarter_note_length)):
            if i == 0:
                inputs.append([128])
                note_type.add(frozenset([128]))
            elif i == 1:
                inputs.append([129])
                note_type.add(frozenset([129]))
            elif i == 2:
                inputs.append([130])
                note_type.add(frozenset([130]))
            elif i == 3:
                inputs.append([131])
                note_type.add(frozenset([131]))
    for j in range(16):
        inputs.append([134])   
    return inputs

def append_notes(max_length, part_i_length, part_i_input):
    note_type.add(frozenset([132]))# 132-->note.Rest
    for num_appended_notes in range(max_length - part_i_length):
        part_i_input.append([132])
    return part_i_input

def check_parts_length(parts_length, inputs):
    max_length = max(parts_length)
    for i in parts_index:
        if parts_length[i] < max_length:
            inputs[i] = append_notes(max_length, parts_length[i], inputs[i])
def make_dataset(file_list):
    X = []
    chorale_key_list = []
    for file in tqdm(file_list):
        s = converter.parse(file)
        for k in range(-5,5):
            interval_type, interval_nature = interval.convertSemitoneToSpecifierGeneric(k)
            transposition_interval = interval.Interval(str(interval_nature) + interval_type)
            chorale_tranposed = s.transpose(transposition_interval)
            
            chorale_key = chorale_tranposed.analyze('key')
            chorale_key_list.append(chorale_key)
            inputs = []
            parts_length = []
        
            for i in parts_index:
                part_input = chorale_to_inputs(s,i)
                parts_length.append(len(part_input))
                inputs.append(part_input)
            check_parts_length(parts_length, inputs)
            np_inputs = np.array(inputs)
            X.append(np_inputs)
    #X.reshape(X.shape[1:])
    print(len(X))
    return X, chorale_key_list